In [3]:
import json
from cadlib.sketch import Profile
from cadlib.extrude import Extrude,CADSequence
import numpy as np

In [4]:
def get_extrusion_index(all_stat, item_id):
    """ Get the index of the extrusion gived the id"""
    for tm in all_stat['timeline']:
        if tm['uuid'] == item_id:
            return tm['index']
    return None

In [6]:
f=json.load(open("/Users/ryzenx/Codes/Python/cc3dparser/temp_data/User Library-3DS Cart.artec_prt.json"))

In [7]:
CADSequence.from_dict(f)

Translation False
Translation False
Translation False
Translation False
Translation False
Translation False
Translation False


In [3]:
extrude_entity=f['entities'][-1]

In [4]:
extrude_entity['feature']['extrude']['refAxis']

[{'start': {'x': 0, 'y': 0.00508, 'z': -0.02424755155239188},
  'end': {'x': -0.000254, 'y': 0.00508, 'z': -0.02424755155239188},
  'direction': {'x': -1, 'y': 0, 'z': 0}}]

In [15]:
np.array(list(extrude_entity['feature']['extrude']['refAxis'][0]['start'].values()))

array([ 0.        ,  0.00508   , -0.02424755])

In [6]:
e=Extrude.from_dict(f,-1)[0]

In [48]:
bbox_2d=e.profile.bbox
zeros = np.zeros((bbox_2d.shape[0], 1))  # (N,2)
bbox_3d = np.concatenate([bbox_2d, zeros], axis=1)  # (N,3)
bbox_4d = np.concatenate(
    [bbox_3d, zeros+1], axis=1).reshape(-1, 4, 1)  # (N,4,1)

transformation_matrix_axis = e.sketch_plane.transformation_matrix_axis  # (4,4)
transformation_matrix_axis = np.broadcast_to(
    transformation_matrix_axis, (bbox_4d.shape[0], 4, 4))  # (N,4,4)

bbox_4d_transformed = transformation_matrix_axis@bbox_4d  # (N,4,1)
bbox_4d_transformed = bbox_4d_transformed.reshape(-1, 4)  # (N,4)
bbox_3d = bbox_4d_transformed[:, :3]  # (N,3)

bbox_3d=np.vstack([bbox_3d.min(axis=0),bbox_3d.max(axis=0)])

In [49]:
bbox_3d

array([[ 0.        ,  0.00502455, -0.02653939],
       [ 0.        ,  0.00508   ,  0.02424755]])

In [52]:
l=[e.bbox[0],e.bbox[0],e.bbox[0]]
np.min(l,axis=0)

array([0.        , 0.00502455, 0.02424755])

In [42]:
np.concatenate(l).reshape(-1,2,3)

array([[[ 0.        ,  0.00502455,  0.02424755],
        [ 0.        ,  0.00508   , -0.02653939]],

       [[ 0.        ,  0.00502455,  0.02424755],
        [ 0.        ,  0.00508   , -0.02653939]],

       [[ 0.        ,  0.00502455,  0.02424755],
        [ 0.        ,  0.00508   , -0.02653939]]])

In [12]:
id_="983A000001000000FFFEFF00000000007B010000"

In [13]:
def get_sketch_id(all_stat,sketch_id):
        for tm in all_stat["timeline"]:
            if tm['uuid']==sketch_id:
                return tm['index']
        return None

In [14]:
sket_entity=f['entities'][29]["sketch"]
p=Profile.from_dict(sket_entity)

In [22]:
ones=np.zeros((2,1))
np.concatenate([p.bbox,ones],axis=1)

array([[-0.02424755,  0.00502455,  0.        ],
       [ 0.02653939,  0.00508   ,  0.        ]])

In [21]:
sample_pts=p.sample_points()

In [19]:
normal_cad_plane=sket_entity["refPlane"]["normal"]

In [28]:
x_axis_3d=np.array(sket_entity["transform"]['data'][0:3])
y_axis_3d=np.array(sket_entity["transform"]['data'][3:6])
z_axis_3d=np.array(sket_entity["transform"]['data'][6:9])
origin=np.array([sket_entity["refPlane"]["origin"]["x"], sket_entity["refPlane"]["origin"]["y"], sket_entity["refPlane"]["origin"]["z"]]).reshape(-1,1)
mat=np.vstack([x_axis_3d, y_axis_3d,z_axis_3d])
ones=np.ones((1,4))

In [9]:
def transformation_matrix_axis(x_axis_3d,y_axis_3d,z_axis_3d,translation,iftranslation=False):
    rotation_mat=np.vstack([x_axis_3d, y_axis_3d,z_axis_3d])
    if iftranslation:
        translation_mat=translation.reshape(-1,1)
    else:
        translation_mat=np.zeros_like(translation.reshape(-1,1))
    transformation_mat=np.concatenate([rotation_mat,translation_mat],axis=1)
    ones=np.array([0,0,0,1]).reshape(1,4)
    translation_mat_homogeneous=np.concatenate([transformation_mat,ones])

    return translation_mat_homogeneous

In [10]:
sampled_points=p.sample_points()
sampled_points=sampled_points.reshape(-1,2)
zeros=np.zeros((len(sampled_points),1))
ones=np.ones((len(sampled_points),1))
sampled_points=np.concatenate([sampled_points,zeros,ones],axis=1)

NameError: name 'p' is not defined

In [11]:
sampled_points.shape

NameError: name 'sampled_points' is not defined

In [52]:
transformation_matrix_axis(x_axis_3d,y_axis_3d,z_axis_3d,origin,Tr)@sampled_points[0]

array([0.        , 0.00502455, 0.02424755, 1.        ])

In [63]:
sampled_points.min(axis=0)

array([-0.02424755,  0.00502455,  0.        ,  1.        ])

In [8]:
transformation_matrix_axis(x_axis_3d,y_axis_3d,z_axis_3d,origin)

NameError: name 'transformation_matrix_axis' is not defined

In [71]:
afterTrans=np.matmul(np.broadcast_to(transformation_matrix_axis(x_axis_3d,y_axis_3d,z_axis_3d,origin,True),(192,4,4)),sampled_points.reshape(-1,4,1))

In [72]:
afterTrans=afterTrans.reshape(-1,4)

In [90]:
mat=np.concatenate([mat,origin],axis=1)

In [91]:
np.concatenate([mat,ones])

array([[ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.00508   ],
       [-1.        ,  0.        ,  0.        , -0.02424755],
       [ 1.        ,  1.        ,  1.        ,  1.        ]])

In [61]:
mat

array([list([0, 0, 1]), list([0, 1, 0]), list([-1, 0, 0]),
       {'x': 0, 'y': 0.00508, 'z': -0.02424755155239188}], dtype=object)

In [24]:
normal_source=np.array([0,0,1])
normal_target=np.array([-1,0,0])

In [25]:
costheta=np.dot(normal_source,normal_target)/(np.linalg.norm(normal_source)*np.linalg.norm(normal_target))


In [28]:
np.cross(normal_source,normal_target)

array([ 0, -1,  0])

In [29]:
sintheta=np.sqrt(1-costheta*2)